<a href="https://colab.research.google.com/github/Jiablero/notebooks/blob/master/my_tiny_XOR_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задача:** написать модель простейшей нейросети, способной обучиться и выполнить операцию XOR.

Навеяно задачей из курса: https://stepik.org/lesson/210101/step/13?unit=183578



In [206]:
import random # потребуется для "обучения"

In [207]:
# пишем свой нейрон с пороговой функцией активации

class neuron():
  def __init__(self, weights, bias, signals = None):	
    self.weights = weights
    self.inputs = len(weights)
    self.bias = bias
    self.minweight = -1
    self.maxweight = 1

    # это позволит создать и получить результат работы нейрона в одно действие
    self.output = self.activate(signals) if signals is not None else None


	# пороговая ф-я активации
  def actfunc(self, result):
    if result <= 0: 
      return 0
    else: return 1


  def activate(self, signals):
    result = 0
    signals = [signals] if type(signals) is not list else signals
    for i in range(0, self.inputs):
      result = result + (self.weights[i] * signals[i])
    result = result + self.bias
    return self.actfunc(result)
  

  def fit(self, Xarr, yarr):
    # обучаем, подставляя значения рандомом.
    # плохая идея, с точки зрения оптимизации, но реализовать проще всего
    iternum = 0
    resparr = []
    
    # пока массив ответов не совпадет с нужными выходными значениями
    # задача простая, и это возможно
    while resparr != yarr:
      resparr = []
      for i in range(0, self.inputs):
        self.weights[i] = random.randint(self.minweight, self.maxweight)
      self.bias = random.randint(self.minweight, self.maxweight)
      for X in Xarr:
        resparr.append(self.activate(X))
      iternum += 1
    print('Fitting ended. Weights:', self.weights, 'Bias:', self.bias, 'Iterations:', iternum)




Сначала попробуем обучать 1 нейрон

In [208]:
# neu пустой нейрон с 1 входом, будем делать операцию НЕ на основе данных nonX [[[x1]], и nony [значение на выходе]
nonX = [[0], [1]]
nony = [1, 0]
neu = neuron(weights = [0], bias = 0)

In [209]:
neu.fit(nonX, nony)

Fitting ended. Weights: [-1] Bias: 1 Iterations: 1


In [210]:
# Ура! Получилось! Нейрон ирвертирует полученные значения.
print(neu.activate([1]), neu.activate([0]))

0 1


Теперь придумаем как быть с несколькими нейронами

In [205]:
# пишем вручную "нейросетку" на классе neuron, вручную расставив веса и bias

# все возможные значения для двух входов
values = [[0, 0], [0, 1], [1, 0], [1, 1]]

# генерим "ответы" из готовой xornn на трёх нейронах на основе values
xornny = []
for v in values:
    x1 = v[0]
    x2 = v[1]
    xornn3 = neuron([1, -1], 0, [
										neuron([1, 1], 0, [x1, x2]).output, 
										neuron([1, 1], -1, [x1, x2]).output
								])		
    print(x1, x2, '=', xornn3.output)
    xornny.append(xornn3.output)

0 0 = 0
0 1 = 1
1 0 = 1
1 1 = 0


In [7]:
# Ура, работает. Сохраняем выходные значения этой сети как эталонные. На них будем обучать.
xornny

[0, 1, 1, 0]

Теперь сделать класс нейросети. На входном слое будет n нейронов с одинаковым количеством входов. На выходном один нейрон с количеством входов, равным кол-ву нейронов входного слоя. Внутренних слоев пока не будет.



In [178]:
class NNetwork():
  def __init__(self, input_amount, inputs_per_neu):
    self.minweight = -1
    self.maxweight = 1
    
    self.input_amount = input_amount # кол-во нейронов во входном слое
    self.inputs_per_neu = inputs_per_neu # кол-во входов в один нейрон

    inputs = [0 for _ in range(self.inputs_per_neu)] # нулевые веса по кол-ву входов для входного слоя
    inputs_outer = [0 for _ in range(self.input_amount)] # нулевые веса по количеству нейронов для входов внешнего слоя
    
    #генерим слои
    self.input_layer = [neuron(weights = inputs, bias = 0) for _ in range(self.input_amount)]
    self.outer_layer = neuron(weights = inputs_outer, bias = 0)

  def layer_activate(self, layer, signals):
    result = []
    signals = [signals] if type(signals) is not list else signals
    for i in range(self.input_amount):
      result.append(layer[i].activate(signals[i]))
    return result

  def activate(self, signals):
    layer1_result = self.layer_activate(self.input_layer, signals)
    result = self.outer_layer.activate(layer1_result)
    return result

  def set_neuron_rnd(self, neu):
    # Расстановка весов аналогично тому как это делается в классе нейрона. 
    # По логике туда бы это и перенести.
    for i in range(0, neu.inputs):
      neu.weights[i] = random.randint(self.minweight, self.maxweight)
    neu.bias = random.randint(self.minweight, self.maxweight)

  def fit(self, Xarr, yarr):
    # обучаем, подставляя значения рандомом.
    iternum = 0
    resparr = []
    while resparr != yarr:
      resparr = []
      for i in range(self.input_amount):
        self.set_neuron_rnd(self.input_layer[i])
      self.set_neuron_rnd(self.outer_layer)
      for i in range(len(Xarr)):
        resparr.append(self.activate(Xarr[i]))
      iternum += 1
      #print('Iter:',iternum, 'output:', resparr, 'expected:', yarr)
    print('Fitting ended. Iterations:', iternum)

#  def fit_cor(self, Xarr, yarr):
    # пробуем метод коррекции ошибки
#    iternum = 0
#    resparr = []
#    while resparr != yarr:
#      resparr = []
#      for y in yarr:
#        for X in Xarr:
#          for x in range(len(X)):
#            pass 
# бросил это дело найдя ошибку в топологии (об этом ниже). Слишком сложное решение для данной задачи.


In [179]:
# Создаем "сетку": число входных нейронов 2, число входов в каждом входном нейроне 1
xor_test = NNetwork(2, 1)

In [197]:
# Поскольку все веса по-умолчанию нулевые, то и выдает она всегда 0
xor_test.activate([0,1])

0

In [ ]:
# Пробуем обучить
xor_test.fit(values, xornny) # будет выполняться бесконечно - неправильная топология сети

Долго не мог понять что неправильно, "сетка" отлично работала со всеми выходными значениями кроме: [1, 0, 0, 1] и [0, 1, 1, 0] (последнее, собственно, результат XOR).

Потом выяснилось что ошибка в количестве входов у нейронов входного слоя, их должно быть два на каждом из двух нейронов, чтобы получить элементы НЕ+И, что не может получится с одним входом. 

Правильная топология такова - входной слой из двух нейронов формирует два элемента НЕ+И:
1.   X и не-Y;
2.   не-X и Y;
3.   Все это подается на входы выходного нейрона, реализующего логику ИЛИ. 

Таким образом получается операция XOR.

Другим вариантом может быть решение на 5-ти нейронах, но для нее класс NNetwork придется чуть-чуть перепилить:
1.   не-X;
2.   не-Y;
3.   Нейрон 'И', получающий на вход результат первого нейрона и Y (не-X и Y);
4.   Нейрон 'И', получающий на вход результат второго нейрона и X (не-Y и X);
5.   Все это подается на входы выходного нейрона, реализующего логику ИЛИ.

При правильной топологии все встает на свои места даже с random - обучением. 

Данные values тоже пришлось поменять: в оба входных нейрона должны приходить одни и те же данные

In [198]:
# Делаем сеть с правильной топологией

xor_test_good = NNetwork(2, 2)


In [204]:
# Переделываем входные значения, на два входа, и чтобы на входы обоих нейронов подавалось одно и то же
values_good = [[[0, 0], [0, 0]],
               [[0, 1], [0, 1]],
               [[1, 0], [1, 0]],
               [[1, 1], [1, 1]]
              ] 
xor_test_good.fit(values_good, xornny)

Fitting ended. Iterations: 8


**Ура!**

In [213]:
# Проверяем
for val in values_good:
  print("Входные данные:", val, "Результат:", xor_test_good.activate(val))

Входные данные: [[0, 0], [0, 0]] Результат: 0
Входные данные: [[0, 1], [0, 1]] Результат: 1
Входные данные: [[1, 0], [1, 0]] Результат: 1
Входные данные: [[1, 1], [1, 1]] Результат: 0
